# LinkedIn Web Scraping

This Jupyter Notebook file is dedicated to extracting data from job offers from the Linkedin website. The focus is to search for Data Analyst opportunities only.

In [1]:
# packages
import pandas as pd
import re # regular expressins
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

In [2]:
# some global variables

# inputation
type_job = 'data analyst'
location = 'United States'

In [3]:
# creating a browser object
path = 'C:/Users/faust/chrome-win32/chromedriver.exe'

options = webdriver.ChromeOptions()
options.add_argument('--incognito')
options.add_argument('--start-maximized')

browser = webdriver.Chrome(path, options=options)
browser.get('https://www.linkedin.com/jobs') 

In [4]:
# Open job's page and search jobs

# job search
browser.find_element_by_xpath('/html/body/main/section[2]/div[2]/ul/li[3]/a').click()
browser.find_element_by_xpath('/html/body/header/nav/ul/li[4]/a').click()

# choose the option jobs
browser.find_element_by_xpath('/html/body/div/header/nav/section/div[1]/button').click()
# browser.find_element_by_xpath('/html/body/div[1]/header/nav/section/div[1]/div[2]').click()

# type job type
browser.find_element_by_xpath('/html/body/div[1]/header/nav/section/section[2]/form/section[1]').click()
job_search = browser.find_element_by_xpath('/html/body/div[1]/header/nav/section/section[2]/form/section[1]/input')
job_search.send_keys(type_job)

# type location
browser.find_element_by_xpath('/html/body/div[1]/header/nav/section/section[2]/form/section[2]/button').click()
browser.find_element_by_xpath('/html/body/div[1]/header/nav/section/section[2]/form/section[2]').click()
location_search = browser.find_element_by_xpath('/html/body/div[1]/header/nav/section/section[2]/form/section[2]/input')
location_search.send_keys(location)

# click on search
browser.find_element_by_xpath('/html/body/div[1]/header/nav/section/section[2]/form/button').click()

In [5]:
# dectecting all jobs
for i in range(100):    
    browser.execute_script("window.scrollTo(0,0)")
    browser.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    try:
#       click in the button "See more jobs"
        browser.find_element_by_xpath('/html/body/div[1]/div/main/section[2]/button').click()
    except:
        pass
    time.sleep(3)

In [6]:
# find the number of jobs
job_lists = browser.find_element_by_class_name('jobs-search__results-list')
jobs = list(job_lists.find_elements_by_tag_name('li')) # return a list
num_of_jobs = len(jobs)
print(f'We consider {num_of_jobs} job offers.')

We consider 1000 job offers.


In [7]:
# load job datails into DataFrame
job_id= []
job_title = []
company_name = []
location = []
date = []
job_link = []

for i in range(num_of_jobs):
# for job in jobs:
 
    job_title0 = jobs[i].find_element_by_css_selector('h3').get_attribute('innerText')
    job_title.append(job_title0)
 
    company_name0 = jobs[i].find_element_by_css_selector('h4').get_attribute('innerText')
    company_name.append(company_name0)
 
    meta_data = jobs[i].find_element_by_class_name('base-search-card__metadata')
    location0 = meta_data.find_element_by_class_name('job-search-card__location').get_attribute('innerText')
    location.append(location0)
 
    date0 = jobs[i].find_element_by_css_selector('div>div>time').get_attribute('datetime')
    date.append(date0)
 
    job_link0 = jobs[i].find_element_by_css_selector('a').get_attribute('href')
    job_link.append(job_link0)

In [8]:
# print(f'job_id size list is {len(job_id)}')
print(f'job_title size list is {len(job_title)}')
print(f'company_name size list is {len(company_name)}')
print(f'location size list is {len(location)}')
print(f'date size list is {len(date)}')
print(f'job_link size list is {len(job_link)}')

job_title size list is 1000
company_name size list is 1000
location size list is 1000
date size list is 1000
job_link size list is 1000


In [9]:
# extract data from each job offer. 
jd = []
seniority = []
emp_type = []
job_func = []
industries = []

step_check = []

for item in range(1, num_of_jobs+1):
    
    job_func0=[]
    industries0=[]
    # clicking job to view job details
    job_click_path_list = [
        f'//*[@id="main-content"]/section[2]/ul/li[{item}]/div/a',
        f'//*[@id="main-content"]/section[2]/ul/li[{item}]/a/div[2]/h3'
    ]
    for i in range(len(job_click_path_list)):
        try:
            job_click = browser.find_element_by_xpath(job_click_path_list[i]).click()
            step_check.append('click ok')
            break
        except:
            continue
    
    time.sleep(5) 
    
    # obtain the job description (if it does not exist, skip to the next job offer)
    jd_path_list = [
        '/html/body/div[1]/div/section/div[2]/div/section[1]/div/div/section/div',
        '/html/body/div[1]/div/section/div[2]/div[1]/section[1]/div/div/section/div',
        '/html/body/div[1]/div/section/div[2]/div/section[2]/div/div/section/div'
    ]
    for i in range(len(jd_path_list)):
        try:
            jd0 = browser.find_element_by_xpath(jd_path_list[i]).get_attribute('innerText')
            jd.append(jd0)
            step_check.append('jd ok')
            break
        except:
            continue
    
    # add seniority, job type, job function, industries
    for i in range(1,5):
        try:
            category_path = f'/html/body/div[1]/div/section/div[2]/div/section[1]/div/ul/li[{i}]/h3'
            info_job_path = f'/html/body/div[1]/div/section/div[2]/div/section[1]/div/ul/li[{i}]/span'
            category = browser.find_element_by_xpath(category_path).get_attribute('innerText')
            info_job = browser.find_element_by_xpath(info_job_path).get_attribute('innerText')
        except:
            continue
        
        if category == 'Seniority level':
            seniority0 = info_job
        
        if category == 'Employment type':
            emp_type0 = info_job
        
        if category == 'Job function':
            job_func_elements0 = info_job
        
        if category == 'Industries':
            industries_elements = info_job

    try:
        seniority.append(seniority0)
        step_check.append('seniority ok')
    except:
        seniority.append('unknown')
        step_check.append('seniority ok')
        
    try:
        emp_type.append(emp_type0)
        step_check.append('type ok')
    except:
        emp_type.append('unknown')
        step_check.append('type ok')
    
    try:
        job_func.append(job_func_elements0)
        step_check.append('job_func ok')
    except:
        job_func.append('unknown')
        step_check.append('job_func ok')
           
    try:
        industries.append(industries_elements)
        step_check.append('industries ok')
    except:
        industries.append('unknown')
        step_check.append('industries ok')
    
    if step_check != ['click ok', 'jd ok', 'seniority ok', 'type ok', 'job_func ok', 'industries ok']:
        print(step_check)
        print(f'Problem with step {item}')
        break

    step_check = []

In [10]:
print(len(jd))

1000


In [11]:
print(f'jd size list is {len(jd)}')
print(f'seniority size list is {len(seniority)}')
print(f'emp_type size list is {len(emp_type)}')
print(f'job_func size list is {len(job_func)}')
print(f'industries size list is {len(industries)}')

jd size list is 1000
seniority size list is 1000
emp_type size list is 1000
job_func size list is 1000
industries size list is 1000


In [12]:
job_data =pd.DataFrame({
    'Date': date,
    'Company': company_name,
    'Job Title': job_title,
    'Location': location,
    'Description': jd,
    'Seniority': seniority,
    'Type': emp_type,
    'Job Function': job_func,
    'Industry': industries,
    'Link': job_link
})

In [13]:
job_data.to_csv('job_data.csv', index = False, header=True)